[Podatkovno zbirko](https://www.clarin.si/repository/xmlui/handle/11356/1398) z označenim sovražnim govorom v objavah s Twitterja smo pridobili s portala Clarin.si V prvem delu se lotimo pridobitve besedila vseh objav v podatkovni zbirki, saj so v prvotni obliki twitti identificirani le z ID-jem in ne s celotnim besedilom objave. V drugem delu se lotimo preprocesiranja podatkov.

## Pridobivanje podatkov s pomočjo Twitter API-ja Tweepy
Na portalu Twitter developer smo si ustvarili račun in pridobili ključe za uporabo Tweepy API-ja.
### Tweepy API ključi

In [1]:
import tweepy

In [2]:
consumer_key = 'tJJWBrVL5ePKzrNMd0upUNCkx'
consumer_secret = 'wpb4Jx04ZufeZCIAdpLqSZfpIFO01OvU6F2jwM5QxoL8uDPKC2'
access_token = '1549038171714998272-4176ZxEc2amIJWhBJMM8NtstEzWBoC'
access_token_secret = 'GQP0HFsWTYN8JEbr9yK47ZJMCzGBHkUsyuatZLsszoPuA'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

Naložimo datoteko, pridobljeno s portala Clarin.

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving IMSyPP_SI_anotacije_training-clarin.csv to IMSyPP_SI_anotacije_training-clarin.csv
User uploaded file "IMSyPP_SI_anotacije_training-clarin.csv" with length 4481908 bytes


### Shranjevanje podatkovne zbirke sovražnega govora v seznam

In [1]:
import csv
corpus = []

with open('IMSyPP_SI_anotacije_training-clarin.csv', 'r', encoding='utf-8') as f:
    lineReader = csv.reader(f, delimiter=',', quotechar="\"")
    for row in lineReader:
        if row:
            #print(row)
            corpus.append({'ID': row[0], 'hateSpeech': row[1]})

In [2]:
corpus = corpus[1:]

In [3]:
len(corpus)

100000

In [4]:
corpus[1]

{'ID': '1130968110834606080', 'hateSpeech': '2 žalitev'}

### Pridobivanje besedila objav s Twitterja
Zapise, ki jih pridobimo s pomočjo Tweepy API-ja, sproti shranjujemo v seznam.

In [ ]:
t = ''
i = 0

with open('hateSpeechTweet.txt', 'w'):
    for tweet in corpus:
        id = tweet['ID']
        #govor = tweet['hateSpeech']
        #print(id)
        try:
            fetched = api.get_status(id)
            t = fetched.text

            if "\n" in t:
              t = t.replace("\n", " ")

            #print(i, t, id)

        except tweepy.TweepError as e: 
            t = e
            #print(i, t, id)

        i += 1
        #tweetsSlo.append({'id': id, 'text': t})
        f.write(i, t, id)

In [5]:
tweets = []

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hateSpeechTweet.txt to hateSpeechTweet.txt
User uploaded file "hateSpeechTweet.txt" with length 12806919 bytes


In [6]:
import re
import csv

with open('hateSpeechTweet.txt', 'r', encoding='utf-8') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  for row in lineReader:
    if row:
      row = ' '.join(row)
      tweets.append(row)

In [7]:
len(tweets)

98466

In [8]:
import pandas as pd
df = pd.DataFrame(tweets, index=None, columns=['Tweet'])

In [9]:
df.shape

(98466, 1)

### Preprocesiranje in čiščenje besedila zbirke
Poiščemo indekse vseh vrstic, ki namesto besedil objav s Twitterja vsebujejo besedila napak: doseženih limitov pridobljenih twittov ali napak, ki povedo, da račun objavljenega twitta ne obstaja več. Besedilo pretvorimo v majhne črke ter odstranimo morebitne dodatne presledke. Odstranimo vrstice, ki vsebujejo napake o doseženem limitu.

In [10]:
s = "Rate limit reached"
limiti = []
errorji = []

for i in range(len(df['Tweet'])):
  #print(i)
  elementi = df['Tweet'][i].split()
  if s in df['Tweet'][i]:
    limiti.append(i)
    df['Tweet'][i-1] += " " + df['Tweet'][i]
  elif elementi[1] == "[{'code':":
    errorji.append(i)

In [11]:
len(limiti)

96

In [12]:
len(errorji)

20052

In [13]:
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join(x.lower().split()))

In [14]:
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join(x.split()))

In [15]:
df.drop(limiti,0,inplace=True)

C:\Users\gogi1\AppData\Local\Temp\ipykernel_12336\3119456342.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(limiti,0,inplace=True)


In [16]:
df.head(10)

,Tweet
0,0 [{'code': 144 'message': 'no status found wi...
1,1 @vitomilz @lukavalas ker poslanci v slovensk...
2,2 @studiocity_ @suzanalovec v novinarstvu je m...
3,3 moja reportaza iz severne koreje:travmatizir...
4,4 [{'code': 144 'message': 'no status found wi...
5,5 @petra_jansa limbarska gora. podpisa ne bo. ...
6,6 [{'code': 144 'message': 'no status found wi...
7,7 [{'code': 144 'message': 'no status found wi...
8,8 @bojanpozar @vladars poklon sodniku! edino o...
9,9 @dixitrader @romanvodeb @bo7ksar kaj je neub...


Tabeli dodamo stolpec z označbo vrste sovražnega govora v objavi.

In [17]:
hateSpeech = []
for tweet in corpus:
  hs = tweet['hateSpeech']
  hateSpeech.append(hs)

In [18]:
len(hateSpeech)

100000

In [19]:
hateSpeech = hateSpeech[:df.shape[0]]

In [20]:
df['HateSpeech'] = hateSpeech

In [21]:
len(df['Tweet'])

98370

In [22]:
len(df['HateSpeech'])

98370

Iz posamezne vrstice v besedilu twitta odstranimo število na začetku. Nato odstranimo tudi ID na koncu posamezne vrstice ter izbrišemo vse vrstice, ki vsebujejo besedila o doseženih napakah: uporabnik ne obstaja, uporabnik je bil blokiran, objava ne obstaja.

In [23]:
tweets = []

for tweet in df['Tweet']:
  elementi = tweet.split()
  #print(elementi)
  if elementi[0].isdigit():
    elementi = elementi[1:]
    tweets.append(' '.join(elementi))
  else:
    tweets.append(tweet)

In [24]:
df['Tweet'] = tweets

In [25]:
df.head(15)

,Tweet,HateSpeech
0,[{'code': 144 'message': 'no status found with...,0 ni sporni govor
1,@vitomilz @lukavalas ker poslanci v slovenskem...,2 žalitev
2,@studiocity_ @suzanalovec v novinarstvu je moč...,2 žalitev
3,moja reportaza iz severne koreje:travmatiziran...,0 ni sporni govor
4,[{'code': 144 'message': 'no status found with...,0 ni sporni govor
5,@petra_jansa limbarska gora. podpisa ne bo. ce...,0 ni sporni govor
6,[{'code': 144 'message': 'no status found with...,0 ni sporni govor
7,[{'code': 144 'message': 'no status found with...,2 žalitev
8,@bojanpozar @vladars poklon sodniku! edino oro...,0 ni sporni govor
9,@dixitrader @romanvodeb @bo7ksar kaj je neubra...,2 žalitev


In [26]:
tweets = []

for tweet in df['Tweet']:
  elementi = tweet.split()
  elementi = elementi[:-1]
  tweets.append(' '.join(elementi))

In [27]:
df['Tweet'] = tweets

In [28]:
df.drop(errorji,0,inplace=True)

C:\Users\gogi1\AppData\Local\Temp\ipykernel_12336\3663003353.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(errorji,0,inplace=True)


In [29]:
df.head(15)

,Tweet,HateSpeech
1,@vitomilz @lukavalas ker poslanci v slovenskem...,2 žalitev
2,@studiocity_ @suzanalovec v novinarstvu je moč...,2 žalitev
3,moja reportaza iz severne koreje:travmatiziran...,0 ni sporni govor
5,@petra_jansa limbarska gora. podpisa ne bo. ce...,0 ni sporni govor
8,@bojanpozar @vladars poklon sodniku! edino oro...,0 ni sporni govor
9,@dixitrader @romanvodeb @bo7ksar kaj je neubra...,2 žalitev
10,@dominikasvarc po mojem sploh ni opazil da nek...,2 žalitev
11,pešci naj posebej pazijo na “železne zavese” k...,0 ni sporni govor
13,@pengovsky @lculiberg @katarinajenko @miharejc...,2 žalitev
14,@bojan__bozic @ajitamxy @petra_jansa morda zat...,2 žalitev


Iz besedil twittov odstranimo vse *@označbeProfilov*, *#ključnike*, elektronsko pošto, URL naslove, *retweete* ter vse posebne simbole

In [30]:
#remove mentions
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'#([a-zA-Z0-9_]{1,50})', '', x).strip())

In [31]:
#remove hashtags
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'@([a-zA-Z0-9_]{1,50})', '', x).strip())

In [32]:
#remove emails
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)', "", x))

In [33]:
#remove URLs
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', "", x))

In [34]:
#remove RTs
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'\brt\b', '', x).strip())

In [35]:
#remove special characters and punctuation
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'[^\w ]+', "", x))

In [36]:
#remove multiple spaces
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join(x.split()))

In [37]:
df = df.reset_index(drop=True)

In [38]:
df.head(30)

,Tweet,HateSpeech
0,ker poslanci v slovenskem parlamentu prav nič ...,2 žalitev
1,v novinarstvu je močan kruhoborski boja bi lah...,2 žalitev
2,moja reportaza iz severne korejetravmatizirani...,0 ni sporni govor
3,limbarska gora podpisa ne bo cerkev se ne prem...,0 ni sporni govor
4,poklon sodniku edino orožje je seznaniti javno...,0 ni sporni govor
5,kaj je neubranljivo partizanski boj za komuniz...,2 žalitev
6,po mojem sploh ni opazil da nekdo za njim drži...,2 žalitev
7,pešci naj posebej pazijo na železne zavese ki ...,0 ni sporni govor
8,državne banke pa so zajebale zakaj ker so reše...,2 žalitev
9,morda zato ker je to storil kot ena debela rit...,2 žalitev


Preverimo, če obstajajo vrstice, ki imajo vrednost v stolpcu *Tweet* ali *HateSpeech* prazno vrednost in take vrstice odstranimo.

In [39]:
prazniHS = []
for i in range(len(df['HateSpeech'])):
  hs = df['HateSpeech'][i]
  if not hs:
    prazniHS.append(i)

In [40]:
len(prazniHS)

144

In [41]:
df.drop(prazniHS,0,inplace=True)

C:\Users\gogi1\AppData\Local\Temp\ipykernel_12336\3001810364.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(prazniHS,0,inplace=True)


In [42]:
df = df.reset_index(drop=True)

In [43]:
prazniText = []
for i in range(len(df['Tweet'])):
  t = df['Tweet'][i]
  if not t:
    prazniText.append(i)

In [44]:
prazniText

[7184, 19612, 20046, 71305]

In [45]:
df.drop(prazniText,0,inplace=True)

C:\Users\gogi1\AppData\Local\Temp\ipykernel_12336\2355411362.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(prazniText,0,inplace=True)


In [46]:
df = df.reset_index(drop=True)

In [47]:
df.shape

(78170, 2)

In [49]:
hatespeech=[]
for hs in df['HateSpeech']:
    if hs == '0 ni sporni govor':
        hs = 'nesovrazni'
        hatespeech.append(hs)
        #print(hs)
    else:
        hs = 'sovrazni'
        hatespeech.append(hs)
        #print(hs)

In [50]:
df['HateSpeech'] = hatespeech

In [51]:
df.head(30)

,Tweet,HateSpeech
0,ker poslanci v slovenskem parlamentu prav nič ...,sovrazni
1,v novinarstvu je močan kruhoborski boja bi lah...,sovrazni
2,moja reportaza iz severne korejetravmatizirani...,nesovrazni
3,limbarska gora podpisa ne bo cerkev se ne prem...,nesovrazni
4,poklon sodniku edino orožje je seznaniti javno...,nesovrazni
5,kaj je neubranljivo partizanski boj za komuniz...,sovrazni
6,po mojem sploh ni opazil da nekdo za njim drži...,sovrazni
7,pešci naj posebej pazijo na železne zavese ki ...,nesovrazni
8,državne banke pa so zajebale zakaj ker so reše...,sovrazni
9,morda zato ker je to storil kot ena debela rit...,sovrazni


Tabelo razdelimo na tri dele v razmerju 8:1:1, ki predstavljajo *train dataset*, *evaluation dataset* in *test dataset*.

In [52]:
test = df[:1000]

In [53]:
test.shape

(1000, 2)

In [54]:
test.head(15)

,Tweet,HateSpeech
0,ker poslanci v slovenskem parlamentu prav nič ...,sovrazni
1,v novinarstvu je močan kruhoborski boja bi lah...,sovrazni
2,moja reportaza iz severne korejetravmatizirani...,nesovrazni
3,limbarska gora podpisa ne bo cerkev se ne prem...,nesovrazni
4,poklon sodniku edino orožje je seznaniti javno...,nesovrazni
5,kaj je neubranljivo partizanski boj za komuniz...,sovrazni
6,po mojem sploh ni opazil da nekdo za njim drži...,sovrazni
7,pešci naj posebej pazijo na železne zavese ki ...,nesovrazni
8,državne banke pa so zajebale zakaj ker so reše...,sovrazni
9,morda zato ker je to storil kot ena debela rit...,sovrazni


In [55]:
evaluation = df[1001:2001]

In [56]:
evaluation.shape

(1000, 2)

In [57]:
evaluation.reset_index(drop=True)

,Tweet,HateSpeech
0,po moje so že tam in so pred časom k nam prile...,sovrazni
1,to je lahk človk še ne vem kolk depressed ko g...,sovrazni
2,in ne zgolj to po trejlerju sodeč je lisbeth s...,nesovrazni
3,pravi povolilni golaž cmarim že 4 ure jutri bo...,nesovrazni
4,levice so polna usta človečnostiahmad za stoti...,nesovrazni
...,...,...
995,nategunska zdravila brez gso kam že to prijavi...,sovrazni
996,migrantje so pičkim dim proti tem mlečno zobim...,sovrazni
997,tuki ji moraš samo rečt da okej ti boš pa tut ...,sovrazni
998,ta usrane ni imel ne čutarice ne cigaretne doz...,sovrazni


In [58]:
train = df[2002:10002]

In [59]:
train.shape

(8000, 2)

In [60]:
evaluation.reset_index(drop=True)

,Tweet,HateSpeech
0,po moje so že tam in so pred časom k nam prile...,sovrazni
1,to je lahk človk še ne vem kolk depressed ko g...,sovrazni
2,in ne zgolj to po trejlerju sodeč je lisbeth s...,nesovrazni
3,pravi povolilni golaž cmarim že 4 ure jutri bo...,nesovrazni
4,levice so polna usta človečnostiahmad za stoti...,nesovrazni
...,...,...
995,nategunska zdravila brez gso kam že to prijavi...,sovrazni
996,migrantje so pičkim dim proti tem mlečno zobim...,sovrazni
997,tuki ji moraš samo rečt da okej ti boš pa tut ...,sovrazni
998,ta usrane ni imel ne čutarice ne cigaretne doz...,sovrazni


Pridobljeno prečiščeni tabeli za *training* in *test dataset* shranimo v novi csv datoteki.

In [61]:
train.to_csv('hateSpeechTrain.csv', encoding = 'utf-8-sig', index=False) 

In [62]:
test.to_csv('hateSpeechTest.csv', encoding = 'utf-8-sig', index=False) 

In [63]:
evaluation.to_csv('hateSpeechEvaluation.csv', encoding = 'utf-8-sig', index=False)